<a href="https://colab.research.google.com/github/manikanta-eng/HPC/blob/main/assignment_6_hpc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Task 1 : Identification of NUMA Architecture and System
Topology**

In [14]:
!lscpu  | grep NUMA

NUMA node(s):                            1
NUMA node0 CPU(s):                       0,1


In [15]:
!apt-get update
!apt-get install -y numactl


Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://cli.github.com/packages stable InRelease [3,917 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [85.0 kB]
Get:8 https://cli.github.com/packages stable/main amd64 Packages [355 B]
Get:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:10 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [9,756 kB]
Get:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease [24.6 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:13 http://security.ubuntu.com/ubuntu jammy-security/multiverse amd64 Packages [62.6 kB]
Get

In [17]:
!numactl --hardware

available: 1 nodes (0)
node 0 cpus: 0 1
node 0 size: 12975 MB
node 0 free: 7254 MB
node distances:
node   0 
  0:  10 


In [18]:
!lscpu

Architecture:                x86_64
  CPU op-mode(s):            32-bit, 64-bit
  Address sizes:             46 bits physical, 48 bits virtual
  Byte Order:                Little Endian
CPU(s):                      2
  On-line CPU(s) list:       0,1
Vendor ID:                   GenuineIntel
  Model name:                Intel(R) Xeon(R) CPU @ 2.20GHz
    CPU family:              6
    Model:                   79
    Thread(s) per core:      2
    Core(s) per socket:      1
    Socket(s):               1
    Stepping:                0
    BogoMIPS:                4399.99
    Flags:                   fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pg
                             e mca cmov pat pse36 clflush mmx fxsr sse sse2 ss h
                             t syscall nx pdpe1gb rdtscp lm constant_tsc rep_goo
                             d nopl xtopology nonstop_tsc cpuid tsc_known_freq p
                             ni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2ap
                   

**Task 2 Memory Bandwidth Measurement on Single NUMA Node**

In [19]:
import numpy as np
import time
N = 100_000_000
a = np.ones(N, dtype=np.float64)
b = np.ones(N, dtype=np.float64)
start = time.time()
c = a + b
end = time.time()
execution_time = end - start
bytes_accessed = 3 * a.nbytes
bandwidth = bytes_accessed / execution_time / 1e9
print("Execution Time:", execution_time, "seconds")
print("Memory Bandwidth:", bandwidth, "GB/s")


Execution Time: 0.3321418762207031 seconds
Memory Bandwidth: 7.225827791751369 GB/s


In [20]:
!numactl --cpunodebind=0 --membind=0 python3 benchmark.py

Execution Time: 0.4207940101623535 seconds
Memory Bandwidth: 5.7035032392072695 GB/s


**task 3 Local vs Remote Memory Access Performance (NUMA-
Aware Execution)**

In [21]:
import numpy as np
import time

N = 100_000_000


a = np.ones(N, dtype=np.float64)
b = np.ones(N, dtype=np.float64)

start = time.time()


c = a + b
end = time.time()

execution_time = end - start


bytes_accessed = 3 * a.nbytes

bandwidth = bytes_accessed / execution_time / 1e9

print("Execution Time:", execution_time, "seconds")
print("Memory Bandwidth:", bandwidth, "GB/s")


Execution Time: 0.3236222267150879 seconds
Memory Bandwidth: 7.416054281441316 GB/s


**task 4 Impact of Multi-threaded Memory Access on NUMA
Systems**

In [22]:
import numpy as np
import threading
import time

N = 50_000_000
a = np.ones(N)

def worker():
    global a
    a = a * 2

threads = []

start = time.time()

for i in range(4):
    t = threading.Thread(target=worker)
    threads.append(t)
    t.start()

for t in threads:
    t.join()

end = time.time()

print("Execution Time:", end - start, "seconds")


Execution Time: 0.6821901798248291 seconds


**task 5 Title  Comparative Study of NUMA-Unaware vs NUMA-Aware
Execution**

In [23]:
import numpy as np
import threading
import time

N = 50_000_000
a = np.ones(N)

def worker():
    global a
    a = a * 2

threads = []

start = time.time()

for i in range(4):
    t = threading.Thread(target=worker)
    threads.append(t)
    t.start()

for t in threads:
    t.join()

end = time.time()

print("Execution Time:", end - start, "seconds")


Execution Time: 0.44888877868652344 seconds


In [24]:
!numactl --cpunodebind=0 --membind=0 python3 5task.py

Execution Time: 0.5426056385040283 seconds
